In [1]:
import torch
from torch.nn import Linear, LayerNorm, Sequential, BatchNorm1d, ModuleList
from torch_geometric.datasets import IMDB
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import LRGBDataset
from torch_geometric.data import DataLoader
from torch_geometric.nn import GPSConv, GATv2Conv, GATConv, GINConv, GCNConv, global_add_pool, global_mean_pool
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch_geometric.nn import global_mean_pool
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
# Load datasets
cora_dataset = Planetoid(root='./data/Cora', name='Cora', split='public')
imdb_dataset = TUDataset(root='./data/IMDB-BINARY', name='IMDB-BINARY')
enzyme_dataset = TUDataset(root='./data/Enzyme', name='ENZYMES')
PascalVOC_dataset = LRGBDataset(root='/tmp/PascalVOC-SP', name='PascalVOC-SP')

In [3]:
### Cora Dataset

In [4]:
## GCN Model

In [5]:
cora_data = cora_dataset[0]
num_output_ch = 16
learning_rate = 0.01

class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, out_channels)
        self.conv2 = GCNConv(out_channels, cora_dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = GCN(cora_dataset.num_node_features, num_output_ch)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
# train the model on Cora
num_epochs = 200
def train(epoch):
    model.train()
    optimizer.zero_grad()
    out = model(cora_data)
    loss = F.nll_loss(out[cora_data.train_mask], cora_data.y[cora_data.train_mask])
    loss.backward()
    optimizer.step()

for epoch in range(num_epochs):
    train(epoch)

# During training, record the accuracy on the validation set.
model.eval()
_, pred = model(cora_data).max(dim=1)
correct = int(pred[cora_data.train_mask].eq(cora_data.y[cora_data.train_mask]).sum().item())
accuracy = correct / cora_data.train_mask.sum().item()
print(f'Train Accuracy: {accuracy:.4f}')

Train Accuracy: 1.0000


In [7]:
# find accuracy on the test set
_, pred = model(cora_data).max(dim=1)
correct = int(pred[cora_data.test_mask].eq(cora_data.y[cora_data.test_mask]).sum().item())
test_accuracy = correct / cora_data.test_mask.sum().item()
print(f'Test Accuracy: {test_accuracy:.4f}')

Test Accuracy: 0.7780


In [8]:
## GIN Model

In [9]:
cora_data = cora_dataset[0]
class GIN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GIN, self).__init__()
        nn1 = torch.nn.Sequential(torch.nn.Linear(in_channels, out_channels), torch.nn.ReLU(), torch.nn.Linear(out_channels, out_channels))
        nn2 = torch.nn.Sequential(torch.nn.Linear(out_channels, out_channels), torch.nn.ReLU(), torch.nn.Linear(out_channels, cora_dataset.num_classes))
        
        self.conv1 = GINConv(nn1)
        self.conv2 = GINConv(nn2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

num_output_ch = 16
learning_rate = 0.01
gin_model = GIN(cora_dataset.num_node_features, num_output_ch)
gin_optimizer = torch.optim.Adam(gin_model.parameters(), lr=learning_rate)

In [10]:
# train the model on Cora
num_epochs = 150
def train():
    gin_model.train()
    gin_optimizer.zero_grad()
    out = gin_model(cora_data)
    loss = F.nll_loss(out[cora_data.train_mask], cora_data.y[cora_data.train_mask])
    loss.backward()
    gin_optimizer.step()
    return loss

def test():
    gin_model.eval()
    logits, accs = gin_model(cora_data), []
    for _, mask in cora_data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(cora_data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

# Training the model
for epoch in range(num_epochs):
    train()
    
train_acc, val_acc, test_acc = test()
print(f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')


Train: 1.0000, Val: 0.7380, Test: 0.7420


In [11]:
## GATv2 Model

In [12]:
class GATv2(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GATv2, self).__init__()
        self.conv1 = GATv2Conv(in_channels, out_channels, heads=1)
        self.conv2 = GATv2Conv(out_channels, cora_dataset.num_classes, heads=1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)
    
gatv2_model = GATv2(cora_dataset.num_node_features, num_output_ch)
gatv2_optimizer = torch.optim.Adam(gatv2_model.parameters(), lr=learning_rate)

In [13]:
def train():
    gatv2_model.train()
    gatv2_optimizer.zero_grad()
    out = gatv2_model(cora_data)
    loss = F.nll_loss(out[cora_data.train_mask], cora_data.y[cora_data.train_mask])
    loss.backward()
    gatv2_optimizer.step()
    return loss

def test():
    gatv2_model.eval()
    logits, accs = gatv2_model(cora_data), []
    for _, mask in cora_data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(cora_data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

for epoch in range(num_epochs):
    train()
    
print(f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')


Train: 1.0000, Val: 0.7380, Test: 0.7420


In [14]:
# Graph GPS

In [15]:
class GPS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, pe_dim=8):
        super().__init__()

        # Use linear layers for node embeddings and positional encodings
        self.node_emb = Linear(in_channels, hidden_channels - pe_dim)
        self.pe_lin = Linear(in_channels, pe_dim)
        self.pe_norm = BatchNorm1d(in_channels)

        self.convs = ModuleList()
        for _ in range(num_layers):
            conv_layer = GCNConv(hidden_channels, hidden_channels)
            self.convs.append(GPSConv(hidden_channels, conv_layer))

        self.mlp = Sequential(
            Linear(hidden_channels, out_channels),
        )

    def forward(self, x, edge_index, batch=None):
        x = self.pe_norm(x)
        x = torch.cat((self.node_emb(x), self.pe_lin(x)), 1)

        for conv in self.convs:
            x = conv(x, edge_index, batch=batch)  # Edge attributes are not passed
        return F.log_softmax(self.mlp(x), dim=1)

In [16]:
model = GPS(in_channels=cora_dataset.num_node_features, hidden_channels=64, num_layers=3, out_channels=cora_dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20,
                              min_lr=0.00001)

In [18]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(cora_data.x, cora_data.edge_index)
    loss = F.nll_loss(out[cora_data.train_mask], cora_data.y[cora_data.train_mask])
    loss.backward()
    optimizer.step()

    pred = out.argmax(dim=1)
    correct = pred[cora_data.train_mask] == cora_data.y[cora_data.train_mask]
    train_accuracy = int(correct.sum()) / int(cora_data.train_mask.sum())
    
    return train_accuracy

def test():
    model.eval()
    out = model(cora_data.x, cora_data.edge_index)
    pred = out.argmax(dim=1)
    correct = pred[cora_data.test_mask] == cora_data.y[cora_data.test_mask]
    return int(correct.sum()) / int(cora_data.test_mask.sum())

num_epochs = 200
final_train_acc, final_test_acc = None, None

for epoch in range(num_epochs):
    train_acc = train()
    test_acc = test()
    scheduler.step(train_acc)  # Note: This should ideally be a loss value

    final_train_acc = train_acc
    final_test_acc = test_acc

# Print the final training and test accuracies
print(f'Final Train Acc: {final_train_acc:.4f}, Final Test Acc: {final_test_acc:.4f}')

Final Train Acc: 1.0000, Final Test Acc: 0.5480


In [19]:
# Enzyme Dataset

In [20]:
# Train test split on Enzyme data
num_examples = len(enzyme_dataset)
num_train = int(0.85 * num_examples)
num_test = num_examples - num_train

dataset = enzyme_dataset.shuffle()
train_dataset = dataset[:num_train]
test_dataset = dataset[num_train:]

# Create dataloaders for train and test data
bs = 64
train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=bs)

/Users/joywang/opt/anaconda3/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [21]:
## GCN Model

In [22]:
class GCNClassifier(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(GCNClassifier, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.fc = torch.nn.Linear(out_channels, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch  # Include batch for batching
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)  # Apply global mean pooling
        x = self.fc(x)
        return x

In [23]:
num_hidden_ch = 16
learning_rate = 0.01
model = GCNClassifier(in_channels=dataset.num_features, hidden_channels=num_hidden_ch, 
                      out_channels=num_output_ch, num_classes=dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
def train(model, loader, optimizer):
    model.train()
    total_loss = 0

    for data in loader:
        optimizer.zero_grad()
        out = model(data)
        loss = F.cross_entropy(out, data.y)  # Cross-entropy loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

num_epochs = 5000
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer)

In [24]:
model.eval()  # Set the model to evaluation mode

# Initialize variables to keep track of correct predictions and total examples
train_correct = 0
train_total = 0

# Validation Set
for data in train_loader:
    with torch.no_grad():  # Disable gradient tracking during evaluation
        out = model(data)  # Forward pass
        _, predicted = out.max(dim=1)  # Get the predicted class labels
        train_total += data.y.size(0)  # Increment the total count
        train_correct += (predicted == data.y).sum().item()  # Increment the correct count
        
train_accuracy = train_correct / train_total
print(f'Train Accuracy: {train_accuracy:.4f}')

Train Accuracy: 0.4745


In [25]:
# Test Set
test_correct = 0
test_total = 0

for data in test_loader:
    with torch.no_grad():  # Disable gradient tracking during evaluation
        out = model(data)  # Forward pass
    _, predicted = out.max(dim=1)  # Get the predicted class labels
    test_total += data.y.size(0)  # Increment the total count
    test_correct += (predicted == data.y).sum().item()  # Increment the correct count
    
test_accuracy = test_correct / test_total

print(f'Test Accuracy: {test_accuracy:.4f}')

Test Accuracy: 0.4111


In [26]:
## GIN Model

In [27]:
class GNN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GNN, self).__init__()
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x):
        x = self.lin(x)
        return F.relu(x)

class GINClassifier(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(GINClassifier, self).__init__()
        self.conv1 = GINConv(GNN(in_channels, hidden_channels))
        self.conv2 = GINConv(GNN(hidden_channels, out_channels))
        self.fc = torch.nn.Linear(out_channels, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return F.log_softmax(x, dim=-1)

In [28]:
num_hidden_ch = 16
learning_rate = 0.01
model = GINClassifier(in_channels=dataset.num_features, hidden_channels=num_hidden_ch, 
                      out_channels=num_output_ch, num_classes=dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
def train(model, loader, optimizer):
    model.train()
    total_loss = 0

    for data in loader:
        optimizer.zero_grad()
        out = model(data)
        loss = F.cross_entropy(out, data.y)  # Cross-entropy loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

num_epochs = 5000
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer)

In [29]:
model.eval()  # Set the model to evaluation mode

# Initialize variables to keep track of correct predictions and total examples
train_correct = 0
train_total = 0

# Validation Set
for data in train_loader:
    with torch.no_grad():  # Disable gradient tracking during evaluation
        out = model(data)  # Forward pass
        _, predicted = out.max(dim=1)  # Get the predicted class labels
        train_total += data.y.size(0)  # Increment the total count
        train_correct += (predicted == data.y).sum().item()  # Increment the correct count
        
train_accuracy = train_correct / train_total
print(f'Train Accuracy: {train_accuracy:.4f}')

Train Accuracy: 0.6059


In [30]:
# Test Set
test_correct = 0
test_total = 0

for data in test_loader:
    with torch.no_grad():  # Disable gradient tracking during evaluation
        out = model(data)  # Forward pass
    _, predicted = out.max(dim=1)  # Get the predicted class labels
    test_total += data.y.size(0)  # Increment the total count
    test_correct += (predicted == data.y).sum().item()  # Increment the correct count
    
test_accuracy = test_correct / test_total

print(f'Test Accuracy: {test_accuracy:.4f}')

Test Accuracy: 0.3444


In [31]:
## GAT2v Model

In [32]:
class GATv2Classifier(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes, heads=1):
        super(GATv2Classifier, self).__init__()
        self.conv1 = GATv2Conv(in_channels, hidden_channels, heads=heads)
        self.conv2 = GATv2Conv(hidden_channels * heads, out_channels, heads=heads)
        self.fc = torch.nn.Linear(out_channels * heads, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return F.log_softmax(x, dim=-1)

In [33]:
model = GATv2Classifier(in_channels=dataset.num_features, hidden_channels=num_hidden_ch, 
                      out_channels=num_output_ch, num_classes=dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
def train(model, loader, optimizer):
    model.train()
    total_loss = 0

    for data in loader:
        optimizer.zero_grad()
        out = model(data)
        loss = F.cross_entropy(out, data.y)  # Cross-entropy loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

num_epochs = 5000
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer)

In [34]:
model.eval()  # Set the model to evaluation mode

# Initialize variables to keep track of correct predictions and total examples
train_correct = 0
train_total = 0

# Validation Set
for data in train_loader:
    with torch.no_grad():  # Disable gradient tracking during evaluation
        out = model(data)  # Forward pass
        _, predicted = out.max(dim=1)  # Get the predicted class labels
        train_total += data.y.size(0)  # Increment the total count
        train_correct += (predicted == data.y).sum().item()  # Increment the correct count
        
train_accuracy = train_correct / train_total
print(f'Train Accuracy: {train_accuracy:.4f}')

Train Accuracy: 0.4373


In [35]:
# Test Set
test_correct = 0
test_total = 0

for data in test_loader:
    with torch.no_grad():  # Disable gradient tracking during evaluation
        out = model(data)  # Forward pass
    _, predicted = out.max(dim=1)  # Get the predicted class labels
    test_total += data.y.size(0)  # Increment the total count
    test_correct += (predicted == data.y).sum().item()  # Increment the correct count
    
test_accuracy = test_correct / test_total

print(f'Test Accuracy: {test_accuracy:.4f}')

Test Accuracy: 0.3333


In [36]:
# Graph GPS

In [37]:
class GraphGPS_ENZYMES(torch.nn.Module):
    def __init__(self, num_features, num_classes, num_layers, hidden_dim, heads, dropout_rate=0.5):
        super().__init__()

        # Input linear transformation
        self.linear = Linear(num_features, hidden_dim)

        # Define new_channels based on hidden_dim and heads
        new_channels = hidden_dim
        self.convs = torch.nn.ModuleList()
        self.norms = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv_layer = GATConv(hidden_dim, hidden_dim, heads=heads, dropout=dropout_rate, concat=False)
            self.convs.append(GPSConv(new_channels, conv_layer))
            self.norms.append(LayerNorm(new_channels))
        
        # Output linear layer
        self.output = Linear(new_channels, num_classes)

    def forward(self, x, edge_index, batch):
        x = self.linear(x)
        
        for conv, norm in zip(self.convs, self.norms):
            x = conv(x, edge_index)
            x = norm(x)  # Apply LayerNorm
            x = F.relu(x)

        x = global_mean_pool(x, batch)
        return F.log_softmax(self.output(x), dim=1)

In [38]:
num_features = enzyme_dataset.num_features
num_classes = enzyme_dataset.num_classes
model = GraphGPS_ENZYMES(num_features, num_classes, num_layers=3, hidden_dim=128, heads=2, dropout_rate=0.5)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training function for GraphGPS_ENZYMES
def train_GPS(model, loader, optimizer):
    model.train()
    for data in loader:
        data = data
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    train_GPS(model, train_dataset, optimizer)

# Evaluation function
def evaluate(model, loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data in loader:
            data = data
            out = model(data.x, data.edge_index, data.batch)
            pred = out.max(dim=1)[1]
            y_true.extend(data.y.cpu().numpy())
            y_pred.extend(pred.cpu().numpy())
    correct = sum(t == p for t, p in zip(y_true, y_pred))
    accuracy = correct / len(y_true)
    return accuracy

# Evaluate the model on the training and test data
train_accuracy = evaluate(model, train_loader)
test_accuracy = evaluate(model, test_loader)
print('Accuracy of GraphGPS on train ENZYMES:', train_accuracy)
print('Accuracy of GraphGPS on test ENZYMES:', test_accuracy)

Accuracy of GraphGPS on train ENZYMES: 0.16666666666666666
Accuracy of GraphGPS on test ENZYMES: 0.16666666666666666


In [39]:
### IBDB dataset

In [40]:
## GCN Model

In [41]:
# Train test split
num_examples = len(imdb_dataset)
num_train = int(0.85 * num_examples)
num_test = num_examples - num_train

dataset = imdb_dataset.shuffle()
train_dataset = dataset[:num_train]
test_dataset = dataset[num_train:]

# Create data loaders for datasets
bs = 64
train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=bs)

In [42]:
class GCNClassifier(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(GCNClassifier, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.fc = torch.nn.Linear(out_channels, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        # Use a constant input feature for all nodes
        x = torch.ones(data.num_nodes, 1)  # Set all input features to 1
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return x

In [43]:
num_hidden_ch = 16
learning_rate = 0.01

# Create the model and optimizer
model = GCNClassifier(in_channels=1, hidden_channels=num_hidden_ch, out_channels=num_hidden_ch, num_classes=imdb_dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
def train(model, loader, optimizer):
    model.train()
    total_loss = 0

    for data in loader:
        optimizer.zero_grad()
        out = model(data)
        loss = F.cross_entropy(out, data.y)  # Cross-entropy loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

# Training loop
num_epochs = 200
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer)

In [44]:
model.eval()  # Set the model to evaluation mode

# Initialize variables to keep track of correct predictions and total examples
train_correct = 0
train_total = 0

# Validation Set
for data in train_loader:
    with torch.no_grad():  # Disable gradient tracking during evaluation
        out = model(data)  # Forward pass
        _, predicted = out.max(dim=1)  # Get the predicted class labels
        train_total += data.y.size(0)  # Increment the total count
        train_correct += (predicted == data.y).sum().item()  # Increment the correct count
        
train_accuracy = train_correct / train_total
print(f'Train Accuracy: {train_accuracy:.4f}')

Train Accuracy: 0.5082


In [45]:
model.eval()  # Set the model to evaluation mode

# Initialize variables to keep track of correct predictions and total examples
test_correct = 0
test_total = 0

for data in test_loader:
    with torch.no_grad():  # Disable gradient tracking during evaluation
        out = model(data)  # Forward pass
    _, predicted = out.max(dim=1)  # Get the predicted class labels
    test_total += data.y.size(0)  # Increment the total count
    test_correct += (predicted == data.y).sum().item()  # Increment the correct count
    
test_accuracy = test_correct / test_total

print(f'Test Accuracy: {test_accuracy:.4f}')

Test Accuracy: 0.4533


In [46]:
## GIN Model

In [47]:

class GNN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GNN, self).__init__()
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x):
        x = self.lin(x)
        return F.relu(x)

class GINClassifier(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(GINClassifier, self).__init__()
        self.conv1 = GINConv(GNN(in_channels, hidden_channels))
        self.conv2 = GINConv(GNN(hidden_channels, out_channels))
        self.fc = torch.nn.Linear(out_channels, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = torch.ones(data.num_nodes, 1)  # Set all input features to 1
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return F.log_softmax(x, dim=-1)

In [48]:
num_hidden_ch = 16
learning_rate = 0.001

# Create the model and optimizer
model = GINClassifier(in_channels=1, hidden_channels=num_hidden_ch, out_channels=num_hidden_ch, num_classes=imdb_dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
def train(model, loader, optimizer):
    model.train()
    total_loss = 0

    for data in loader:
        optimizer.zero_grad()
        out = model(data)
        loss = F.cross_entropy(out, data.y)  # Cross-entropy loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

# Training loop
num_epochs = 2000
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer)

In [49]:
model.eval()  # Set the model to evaluation mode

# Initialize variables to keep track of correct predictions and total examples
train_correct = 0
train_total = 0

# Validation Set
for data in train_loader:
    with torch.no_grad():  # Disable gradient tracking during evaluation
        out = model(data)  # Forward pass
        _, predicted = out.max(dim=1)  # Get the predicted class labels
        train_total += data.y.size(0)  # Increment the total count
        train_correct += (predicted == data.y).sum().item()  # Increment the correct count
        
train_accuracy = train_correct / train_total
print(f'Train Accuracy: {train_accuracy:.4f}')

Train Accuracy: 0.7165


In [50]:
# Test Set
test_correct = 0
test_total = 0

for data in test_loader:
    with torch.no_grad():  # Disable gradient tracking during evaluation
        out = model(data)  # Forward pass
    _, predicted = out.max(dim=1)  # Get the predicted class labels
    test_total += data.y.size(0)  # Increment the total count
    test_correct += (predicted == data.y).sum().item()  # Increment the correct count
    
test_accuracy = test_correct / test_total

print(f'Test Accuracy: {test_accuracy:.4f}')

Test Accuracy: 0.6667


In [51]:
## Gat2v Model

In [52]:
class GATv2Classifier(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes, heads=1):
        super(GATv2Classifier, self).__init__()
        self.conv1 = GATv2Conv(in_channels, hidden_channels, heads=heads)
        self.conv2 = GATv2Conv(hidden_channels * heads, out_channels, heads=heads)
        self.fc = torch.nn.Linear(out_channels * heads, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = torch.ones(data.num_nodes, 1)  # Set all input features to 1
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return F.log_softmax(x, dim=-1)

In [53]:
# Create the model and optimizer
num_hidden_ch = 16
learning_rate = 0.001

model = GATv2Classifier(in_channels=1, hidden_channels=num_hidden_ch, out_channels=num_hidden_ch, num_classes=imdb_dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
def train(model, loader, optimizer):
    model.train()
    total_loss = 0

    for data in loader:
        optimizer.zero_grad()
        out = model(data)
        loss = F.cross_entropy(out, data.y)  # Cross-entropy loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer)

In [54]:
model.eval()  # Set the model to evaluation mode

# Initialize variables to keep track of correct predictions and total examples
train_correct = 0
train_total = 0

# Validation Set
for data in train_loader:
    with torch.no_grad():  # Disable gradient tracking during evaluation
        out = model(data)  # Forward pass
        _, predicted = out.max(dim=1)  # Get the predicted class labels
        train_total += data.y.size(0)  # Increment the total count
        train_correct += (predicted == data.y).sum().item()  # Increment the correct count
        
train_accuracy = train_correct / train_total
print(f'Train Accuracy: {train_accuracy:.4f}')

Train Accuracy: 0.5082


In [55]:
# Test Set
test_correct = 0
test_total = 0

for data in test_loader:
    with torch.no_grad():  # Disable gradient tracking during evaluation
        out = model(data)  # Forward pass
    _, predicted = out.max(dim=1)  # Get the predicted class labels
    test_total += data.y.size(0)  # Increment the total count
    test_correct += (predicted == data.y).sum().item()  # Increment the correct count
    
test_accuracy = test_correct / test_total

print(f'Test Accuracy: {test_accuracy:.4f}')

Test Accuracy: 0.4533


In [56]:
# GraphGPS

In [57]:
class GPS(torch.nn.Module):
    def __init__(self, num_features, num_classes, num_layers, new_channels, dropout_rate=0.5):
        super().__init__()
        self.input_transform = Linear(num_features, new_channels)
        self.convs = torch.nn.ModuleList()
        self.norms = torch.nn.ModuleList()
        
        for _ in range(num_layers):
            conv_layer = GCNConv(new_channels, new_channels)
            self.convs.append(GPSConv(new_channels, conv_layer))
            self.norms.append(LayerNorm(new_channels))

        self.dropout = torch.nn.Dropout(dropout_rate)
        self.output = Linear(new_channels, num_classes)
        
    def forward(self, data):
        def degree_feature(data):
            deg = torch.zeros(data.num_nodes, dtype=torch.float).to(data.edge_index.device)
            deg.index_add_(0, data.edge_index[0], torch.ones(data.edge_index.size(1), device=data.edge_index.device))
            deg.index_add_(0, data.edge_index[1], torch.ones(data.edge_index.size(1), device=data.edge_index.device))
            return deg.view(-1, 1)
        
        const_feat = torch.ones((data.num_nodes, 1))
        deg_feat = degree_feature(data)
        rand_feat = torch.rand((data.num_nodes, 1))

        x = torch.cat([const_feat, deg_feat, rand_feat], dim=1)
        edge_index = data.edge_index
        batch = data.batch

        for conv, norm in zip(self.convs, self.norms):
            x = conv(x, edge_index)
            x = norm(x)
            x = F.relu(x)
            x = self.dropout(x)

        x = global_mean_pool(x, batch)
        return F.log_softmax(self.output(x), dim=1)

In [58]:
model = GPS(num_features=imdb_dataset.num_node_features, num_classes=imdb_dataset.num_classes, 
                         num_layers=5, new_channels=3)

/Users/joywang/opt/anaconda3/lib/python3.9/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [59]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

In [60]:
def train():
    model.train()
    for data in train_loader:
        model.train()
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()

for epoch in range(500):
    train()

model.eval()
correct = 0
for data in train_loader:
    _, pred = model(data).max(dim=1)
    correct += pred.eq(data.y).sum().item()

train_acc = correct / len(train_loader)
print('Accuracy of GPS on train IMDB:', train_acc)

model.eval()
correct = 0
for data in test_loader:
    data = data
    _, pred = model(data).max(dim=1)
    correct += pred.eq(data.y).sum().item()

test_acc = correct / len(test_loader)
print('Accuracy of GPS on test IMDB:', test_acc)

In [ ]:
### LongRange Dataset

In [ ]:
PascalVOC_data = PascalVOC_dataset[0]
num_output_ch = 16
learning_rate = 0.01

num_nodes = PascalVOC_data.num_nodes 
test_percentage = 0.1

indices = torch.randperm(num_nodes)
train_indices = indices[:int(num_nodes * (1 - test_percentage))]
test_indices = indices[int(num_nodes * (1 - test_percentage)):]

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[train_indices] = True
test_mask[test_indices] = True

# Add the train_mask and test_mask to your PascalVOC_data object
PascalVOC_data.train_mask = train_mask
PascalVOC_data.test_mask = test_mask

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, out_channels)
        self.conv2 = GCNConv(out_channels, PascalVOC_dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = GCN(PascalVOC_data.num_node_features, num_output_ch)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# train the model
num_epochs = 200
def train(epoch):
    model.train()
    optimizer.zero_grad()
    out = model(PascalVOC_data)
    loss = F.nll_loss(out[PascalVOC_data.train_mask], PascalVOC_data.y[PascalVOC_data.train_mask])
    loss.backward()
    optimizer.step()

for epoch in range(num_epochs):
    train(epoch)

# During training, record the accuracy on the validation set.
model.eval()
_, pred = model(PascalVOC_data).max(dim=1)
correct = int(pred[PascalVOC_data.train_mask].eq(PascalVOC_data.y[PascalVOC_data.train_mask]).sum().item())
accuracy = correct / PascalVOC_data.train_mask.sum().item()
print(f'Train Accuracy: {accuracy:.4f}')

Train Accuracy: 0.7971


In [ ]:
# find accuracy on the test set
_, pred = model(PascalVOC_data).max(dim=1)
correct = int(pred[PascalVOC_data.test_mask].eq(PascalVOC_data.y[PascalVOC_data.test_mask]).sum().item())
test_accuracy = correct / PascalVOC_data.test_mask.sum().item()
print(f'Test Accuracy: {test_accuracy:.4f}')

Test Accuracy: 0.7609


In [ ]:
## GIN Model

In [ ]:
class GIN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GIN, self).__init__()
        nn1 = torch.nn.Sequential(torch.nn.Linear(in_channels, out_channels), torch.nn.ReLU(), torch.nn.Linear(out_channels, out_channels))
        nn2 = torch.nn.Sequential(torch.nn.Linear(out_channels, out_channels), torch.nn.ReLU(), torch.nn.Linear(out_channels, cora_dataset.num_classes))
        
        self.conv1 = GINConv(nn1)
        self.conv2 = GINConv(nn2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

num_output_ch = 16
learning_rate = 0.01
gin_model = GIN(PascalVOC_data.num_node_features, num_output_ch)
gin_optimizer = torch.optim.Adam(gin_model.parameters(), lr=learning_rate)

In [ ]:
def train():
    gin_model.train()
    gin_optimizer.zero_grad()
    out = gin_model(PascalVOC_data)
    loss = F.nll_loss(out[PascalVOC_data.train_mask], PascalVOC_data.y[PascalVOC_data.train_mask])
    loss.backward()
    gin_optimizer.step()
    return loss

def test():
    gin_model.eval()
    logits, accs = gin_model(PascalVOC_data), []
    for _, mask in PascalVOC_data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(PascalVOC_data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

# Training the model
for epoch in range(num_epochs):
    train()
    
train_acc, val_acc, test_acc = test()
print(f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')


Train: 1.0000, Val: 0.7360, Test: 0.7500


In [ ]:
## GATv2 Model

In [ ]:
class GATv2(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GATv2, self).__init__()
        self.conv1 = GATv2Conv(in_channels, out_channels, heads=1)
        self.conv2 = GATv2Conv(out_channels, cora_dataset.num_classes, heads=1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)
    
gatv2_model = GATv2(PascalVOC_data.num_node_features, num_output_ch)
gatv2_optimizer = torch.optim.Adam(gatv2_model.parameters(), lr=learning_rate)

In [ ]:
def train():
    gatv2_model.train()
    gatv2_optimizer.zero_grad()
    out = gatv2_model(PascalVOC_data)
    loss = F.nll_loss(out[PascalVOC_data.train_mask], PascalVOC_data.y[PascalVOC_data.train_mask])
    loss.backward()
    gatv2_optimizer.step()
    return loss

def test():
    gatv2_model.eval()
    logits, accs = gatv2_model(cora_data), []
    for _, mask in PascalVOC_data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(PascalVOC_data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

for epoch in range(num_epochs):
    train()
    
print(f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')


Train: 1.0000, Val: 0.7360, Test: 0.7500


In [ ]:
# Graph GPS

In [ ]:
class GPS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, pe_dim=8):
        super().__init__()

        # Use linear layers for node embeddings and positional encodings
        self.node_emb = Linear(in_channels, hidden_channels - pe_dim)
        self.pe_lin = Linear(in_channels, pe_dim)
        self.pe_norm = BatchNorm1d(in_channels)

        self.convs = ModuleList()
        for _ in range(num_layers):
            conv_layer = GCNConv(hidden_channels, hidden_channels)
            self.convs.append(GPSConv(hidden_channels, conv_layer))

        self.mlp = Sequential(
            Linear(hidden_channels, out_channels),
        )

    def forward(self, x, edge_index, batch=None):
        x = self.pe_norm(x)
        x = torch.cat((self.node_emb(x), self.pe_lin(x)), 1)

        for conv in self.convs:
            x = conv(x, edge_index, batch=batch)  # Edge attributes are not passed
        return F.log_softmax(self.mlp(x), dim=1)

In [ ]:
model = GPS(in_channels=PascalVOC_data.num_node_features, hidden_channels=64, num_layers=3, out_channels=cora_dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20,
                              min_lr=0.00001)

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(PascalVOC_data.x, PascalVOC_data.edge_index)
    loss = F.nll_loss(out[PascalVOC_data.train_mask], PascalVOC_data.y[PascalVOC_data.train_mask])
    loss.backward()
    optimizer.step()

    pred = out.argmax(dim=1)
    correct = pred[PascalVOC_data.train_mask] == PascalVOC_data.y[PascalVOC_data.train_mask]
    train_accuracy = int(correct.sum()) / int(PascalVOC_data.train_mask.sum())
    
    return train_accuracy

def test():
    model.eval()
    out = model(PascalVOC_data.x, PascalVOC_data.edge_index)
    pred = out.argmax(dim=1)
    correct = pred[PascalVOC_data.test_mask] == PascalVOC_data.y[PascalVOC_data.test_mask]
    return int(correct.sum()) / int(PascalVOC_data.test_mask.sum())

num_epochs = 200
final_train_acc, final_test_acc = None, None

for epoch in range(num_epochs):
    train_acc = train()
    test_acc = test()
    scheduler.step(train_acc)  # Note: This should ideally be a loss value

    final_train_acc = train_acc
    final_test_acc = test_acc

# Print the final training and test accuracies
print(f'Final Train Acc: {final_train_acc:.4f}, Final Test Acc: {final_test_acc:.4f}')